<a href="https://colab.research.google.com/github/anupampani123/Soundeffectgenerator/blob/main/dataset_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import librosa as lr
import matplotlib.pyplot as plt
from glob import glob
import soundfile as sf
import os
import sys
import csv
import librosa.display
import librosa 
import cv2
import soundfile as sf
import glob


In [ ]:
#Unzip the dataset which contains the .wav file 
!unzip "/content/drive/MyDrive/Suga/AIR_wav_files.zip" -d "/content/"

Archive:  /content/drive/MyDrive/Suga/AIR_wav_files.zip
  inflating: /content/air_stairway_1_1_3_180_logsweep.wav  
  inflating: /content/air_stairway_1_1_3_180_mls.wav  
  inflating: /content/air_stairway_1_1_3_135_mls.wav  
  inflating: /content/air_stairway_1_1_3_150_logsweep.wav  
  inflating: /content/air_stairway_1_1_3_150_mls.wav  
  inflating: /content/air_stairway_1_1_3_165_logsweep.wav  
  inflating: /content/air_stairway_1_1_3_165_mls.wav  
  inflating: /content/air_stairway_1_1_3_120_mls.wav  
  inflating: /content/air_stairway_1_1_3_135_logsweep.wav  
  inflating: /content/air_stairway_1_1_3_105_mls.wav  
  inflating: /content/air_stairway_1_1_3_120_logsweep.wav  
  inflating: /content/air_stairway_1_1_3_75_mls.wav  
  inflating: /content/air_stairway_1_1_3_90_logsweep.wav  
  inflating: /content/air_stairway_1_1_3_90_mls.wav  
  inflating: /content/air_stairway_1_1_3_105_logsweep.wav  
  inflating: /content/air_stairway_1_1_3_15_logsweep.wav  
  inflating: /content/air_st

In [ ]:
img_size1=513
img_size2=128

In [ ]:
def plot_specgrams(log_power_spectra, rate, filename, output_dir):
    """ 
    Save log-power and normalized log-power specotrgrams to file.
    Args:
        log_power_spectra (ndarray): Comptued Log-Power spectra.
        rate (int): Sample rate of input audio data.
        filename (str): Output filename for generated plot.
        output_dir (str): Directory to save generated plot.
    """

    plt.figure()
    librosa.display.specshow(log_power_spectra, sr=rate*2, y_axis='log', x_axis='time')
    plt.colorbar(format='%+2.0f dB')
    plt.title('Log-Power spectrogram')
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, filename))
    plt.close('all')

In [ ]:
def fix_specgram_shape(spec, shape=(513,128)):
    """Fix spectrogram shape to user specified size.
    Args:
        spec: 2D spectrogram [freqs, time].
        shape: 2D output spectrogram shape [freqs, time].
    Returns:
        fixed_spec: fixed 2D output spectrogram [freqs, time].
    """
    if spec.shape[1] < shape[1]:
        out = np.zeros(shape)
        out[:spec.shape[0],:spec.shape[1]] = spec
    else: 
        out = spec[:,:shape[1]]  
            
    return out

In [ ]:
filelist = []
path="/content/"
for root, dirs, files in os.walk(path):
  for file in files:
    name=os.path.join(root,file)
    if ".wav" not in name:
      continue
    else:
      filelist.append(name)

NameError: ignored

In [ ]:
def load_data(filelist, split=True, train_split=0.80):
    """ 
    Utility function to load room impulse responses.
    Args:
        dataset_dir (str): Directory containing the dataset.
        train_split (float, optional): Fraction of the data to return as training samples.
    Returns:
        x_train (ndarray): Training examples with shape (examples, audio samples).
        x_test (ndarray): Testing examples with shape (examples, audio samples).	
    """


    IRs = [] # list to hold audio data
    sample_names = [] # temp list - delete this later - maybe not?
    load_samples = 0
    for idx, sample in enumerate(filelist):
        data, rate = sf.read(sample, always_2d=True)

        for ch in range(data.shape[1]):
            IRs.append(data[:,ch])
            load_samples += 1
            sample_names.append(os.path.basename(sample).replace('.wav', ''))

        sys.stdout.write("* Loaded {} RIRs\r".format(load_samples+1))
        sys.stdout.flush()

    if not split:
        return IRs, sample_names

    x = np.stack(IRs, axis=0)
    x = np.reshape(x, (x.shape[0], x.shape[1], 1))

    train_idx = np.floor(n_samples*train_split).astype('int')
    x_train = x[:train_idx,:]
    x_test = x[train_idx:,:]

    print("Loaded data with shape:")
    print("x_train: {}".format(x_train.shape))
    print("x_test:  {}".format(x_test.shape))

    return x_train, x_test

In [ ]:
def generate_specgrams(filelist, 
                       output_dir, 
                       rate=16000, 
                       n_fft=1024, 
                       n_hop=256, 
                       augment_data=False, 
                       save_plots=False):
    """ 
    Generate spectrograms (via stft) on dataset of audio data.
    Args:
        filelist (str): list containing the names of the datasets.
        output_dir (str): Directory to store outputs.
        sequence_len (int): Length of output audio data.
        rate (int, optional): Sample rate out input audio data.
        n_fft (int, optional): Size of the FFT to generate spectrograms.
        n_hop (int, optional): Hop size for FFT.
        augment_data (bool, optional): Generate augmented (stretched and shifted) audio.
        save_plot (bool, optional): Generate plots of spectrograms
    """
    if not os.path.isdir(output_dir):
        os.makedirs(output_dir)
   
    IRs, sample_names = load_data(filelist, split=False)

    specs_generated = 0
    n_specs = len(IRs)

    for idx in range(len(IRs)):
        audio = np.reshape(IRs[idx], (IRs[idx].shape[0],))

        # if augment_data:
            
        #     augmented_audio = augment_audio(audio, 16000, 
        #                                     stretch_factors=[0.80, 0.90, 1.10, 1.20], 
        #                                     shift_factors=[-2, -1, 1, 2])
        #     n_specs = len(IRs) * (len(stretch_factors) + len(shift_factors))                                
        #     for augment in augmented_audio:
        #         S = librosa.stft(augment, n_fft=n_fft, hop_length=n_hop, center=True)
        #         power_spectra = np.abs(S)**2
        #         log_power_spectra = librosa.power_to_db(power_spectra)
        #         _min = np.amin(log_power_spectra)
        #         _max = np.amax(log_power_spectra)
        #         normalized_log_power_spectra = (log_power_spectra - _min) / (_max - _min)
        #         filename = f"ir_{sample_names[idx]}_{specs_generated+1}"
        #         np.savetxt(os.path.join(output_dir, filename + ".txt"), normalized_log_power_spectra)
        #         specs_generated += 1

        #         if save_plots:
        #             if not os.path.isdir("spect_plots"):
        #                 os.makedirs("spect_plots")
        #             plot_specgrams(log_power_spectra, normalized_log_power_spectra, 
        #                               16000, filename + ".png", "spect_plots")
        
        S = librosa.stft(audio, n_fft=n_fft, hop_length=n_hop, center=True)
        power_spectra = np.abs(S)**2
        log_power_spectra = librosa.power_to_db(power_spectra)
        _min = np.amin(log_power_spectra)
        _max = np.amax(log_power_spectra)
        if _min == _max:
            print("divide by zero")
        else:
            normalized_log_power_spectra = (log_power_spectra - _min) / (_max - _min)
            normalized_log_power_spectra= fix_specgram_shape(normalized_log_power_spectra)
            filename = f"ir_{sample_names[idx]}_{specs_generated+1}"
            np.savetxt(os.path.join(output_dir, filename + ".txt"), normalized_log_power_spectra)
            specs_generated += 1

            if save_plots:
                if not os.path.isdir("/content/drive/MyDrive/Suga/spect_plots"):
                    os.makedirs("/content/drive/MyDrive/Suga/spect_plots")
                plot_specgrams(normalized_log_power_spectra, 16000, filename + ".png", "/content/drive/MyDrive/Suga/spect_plots")

        sys.stdout.write(f"* Computed {specs_generated}/{n_specs} RIR spectrograms\r")
        sys.stdout.flush()

In [ ]:
generate_specgrams(filelist,"/content/drive/MyDrive/Suga/spect_text_new",save_plots=False)